In [ ]:
import xml.etree.ElementTree as ET
import pandas as pd
import re

tree = ET.parse(r'\\youtpath\file.twb')
root = tree.getroot()

# parameters

caption_list = []
name_list = []
value_list = []

for child in root.find('.//datasources'):
    for elem in child.iter(tag ='column'):
        caption = elem.get('caption')
        name = elem.get('name')
        parameter = elem.get('param-domain-type')
        if caption is None or parameter is None:
            continue
        member = elem.find('members')
        if member is None:
            ran = elem.find('range')
            if ran is None:
                continue
        caption_list.append(caption)
        name_list.append(name)
        
        # list
        if member is not None:
            temp_list = []
            for elem2 in elem.iter(tag ='member'):
                value = elem2.get('value')
                temp_list.append(value)
            value_list.append(temp_list)
        else:
            temp_list = []
            for elem2 in elem.iter(tag ='range'):
                min_value = elem2.get('min')
                temp_list.append(min_value)
                max_value = elem2.get('max')
                temp_list.append(max_value)               
            value_list.append(temp_list)
            
df = pd.DataFrame(list(zip(caption_list, name_list, value_list)), columns =['name', 'alt_name', 'formula'])
df = df.replace('\r\n', ' ', regex=True)
df.insert(0, 'tableau', 'parameter')
df = df.drop_duplicates(subset='name', keep='first')
df_par = df.copy()

# calculations

caption_list = []
name_list = []
formula_list = []

# for child in root.find('.//worksheets/worksheet/table/view'):
for child in root.find('.//datasources'):
    for elem in child.iter(tag ='column'):
        caption = elem.get('caption')
        name = elem.get('name')
        parameter = elem.get('param-domain-type')
        if caption is None or parameter is not None:
            continue
        calculation = elem.find('calculation')
        if calculation is None:
            continue
        
        caption_list.append(caption)
        name_list.append(name)
        for elem2 in elem.iter(tag ='calculation'):
            formula = elem2.get('formula')
            formula_list.append(formula)
            break

df = pd.DataFrame(list(zip(caption_list, name_list, formula_list)), columns =['name', 'alt_name', 'formula'])
df = df.replace('\r\n', ' ', regex=True)
df.insert(0, 'tableau', 'calculation')
df = df.drop_duplicates(subset='name', keep='first')
df_calc = df.copy()

# appending calculations to parameters

df = df_par.append(df_calc, ignore_index = True)
df['name'] = ('[' + df['name'] + ']')
df['formula'] = df['formula'].astype(str)
for i in range(0, df.shape[0]):
    a = df.iloc[i, 2]
    b = df.iloc[i, 1]
#     print(a)
#     print(b)
    df['formula'] = df['formula'].str.replace(re.escape(a), b, regex=True)
    
print(df.shape)
df.head()

extract = df.to_excel (r'C:\yourpath\export_dataframe.xlsx', index = None, header=True)